# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import os
from glob import glob

price_data = os.environ['PRICE_DATA']
parquet_files = glob(os.path.join(price_data, '**', '*.parquet'), recursive=True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [29]:
df = dd.read_parquet(parquet_files)
df['Close_lag_1'] = df.groupby('ticker')['Close'].shift(1)
df['Adj_Close_lag_1'] = df.groupby('ticker')['Adj Close'].shift(1)
df['returns'] = df['Close'] / df['Close_lag_1'] - 1
df['hi_lo_range'] = df['High'] - df['Low']

# Remove any duplicate columns just in case
df = df.loc[:, ~df.columns.duplicated()]

dd_feat = df.copy()


/tmp/ipykernel_3091/1081290869.py:2: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  df['Close_lag_1'] = df.groupby('ticker')['Close'].shift(1)
/tmp/ipykernel_3091/1081290869.py:3: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  df['Adj_Close_lag_1'] = df.groupby('ticker')['Adj Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [31]:
# # dd_feat = dd_feat.loc[:, ~dd_feat.columns.duplicated()]
# # pd_feat = dd_feat.compute()

# # print(dd_feat.known_divisions)   # shows partition index metadata
# # print(dd_feat.index.compute().is_unique)  # tells if index has dupes

# # print(dd_feat.head(1000).index.is_unique)

# # print(type(dd_feat.index))
# # print(dd_feat.index.name)

# # pd_feat = dd_feat.reset_index(drop=True).compute()

# # pd_feat = dd_feat.reset_index(drop=True).compute()

# # print(dd_feat.npartitions)    
# # 
# # dd_feat = dd_feat.repartition(npartitions=50)  # adjust based on memory

# # pd_feat = dd_feat.reset_index(drop=True).compute()

# # dd_feat["returns_ma10"] = dd_feat["returns"].rolling(10).mean()
# # pd_feat = dd_feat.head(5000)  # convert only a slice

# # # Just take a subset so it doesn't choke
# # pd_feat = dd_feat.head(5000)  

# # # Compute rolling mean in Pandas
# # pd_feat["returns_ma10"] = pd_feat["returns"].rolling(10).mean()

# # print(dd_feat.columns)

# import pandas as pd
# import dask.dataframe as dd

# # Take just the first partition into Pandas (fast, avoids scanning all 2848)
# # pd_feat = dd_feat.get_partition(0).head(1000, compute=True)


# # # Compute 10-day rolling mean of returns
# # pd_feat["returns_ma10"] = pd_feat["returns"].rolling(10).mean()

# # # Show first few rows
# # print(pd_feat[["returns", "returns_ma10"]].head(15))

# # Compute the Dask DataFrame to pandas, limit to first 1000 rows for speed
# # pd_feat = dd_feat.head(1000)

# # # Compute 10-day rolling mean
# # pd_feat["returns_ma10"] = pd_feat["returns"].rolling(10).mean()

# # print(pd_feat.head(15))

# import pandas as pd

# # Read a small sample directly with Pandas
# pd_feat = pd.read_parquet(parquet_files, engine="pyarrow").head(1000)

# # Compute 10-day rolling mean of returns
# pd_feat["returns_ma10"] = pd_feat["returns"].rolling(10).mean()

# print(pd_feat[["returns", "returns_ma10"]].head(15))


import pandas as pd

# Load a manageable slice directly with Pandas
pd_feat = pd.read_parquet(parquet_files, engine="pyarrow").head(1000)

# Recreate the engineered 'returns' column
pd_feat["Close_lag_1"] = pd_feat["Close"].shift(1)
pd_feat["returns"] = pd_feat["Close"] / pd_feat["Close_lag_1"] - 1

# Now compute the 10-day rolling mean of returns
pd_feat["returns_ma10"] = pd_feat["returns"].rolling(10).mean()

print(pd_feat[["returns", "returns_ma10"]].head(15))


                      returns  returns_ma10
__null_dask_index__                        
704                       NaN           NaN
705                 -0.004244           NaN
706                  0.006926           NaN
707                  0.005291           NaN
708                 -0.010526           NaN
709                  0.010638           NaN
710                  0.000000           NaN
711                  0.000000           NaN
712                  0.002632           NaN
713                  0.002100           NaN
714                  0.021477      0.003429
715                  0.000000      0.003854
716                  0.000000      0.003161
717                  0.000000      0.002632
718                  0.012821      0.004967


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

I have lost track of the point of this assignment, because all I've done, for hours a day, for the past two weeks, is desperately attempt to satisfy the instructions. 

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.